In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [3]:
city_list = city_link_collector()
len(city_list)

415

In [16]:
def craigslist_motorcycle_scraper(city_list):

    title = []
    price = []
    neighborhood = []
    attributes = []
    description = []
    city_names = []

    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            title_ = []
            price_ = []
            neighborhood_ = []
            attributes_ = []
            description_ = []
            city_names_ = []
            #get request      
            response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, response.status_code))

            #define the html text
            post_soup = BeautifulSoup(response.text, 'html.parser')

            #define the posts
    #         posts = page_html.find_all('li', class_= 'result-row')

            #extract data by item
    #         for post in posts:
            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                sub_post = requests.get(link)
                sub_soup = BeautifulSoup(sub_post.text, 'html.parser')

    #             if tag.find('small') is not None:

                post_title = sub_soup.find('span', id = 'titletextonly')
                title_.append(post_title)

                post_price = sub_soup.find('span', class_ = 'price')
                price_.append(post_price)

                post_neighborhood = sub_soup.find('small')
                neighborhood_.append(post_neighborhood)

                post_attributes = sub_soup.find_all('p', attrs = {'class': 'attrgroup'})
                attributes_.append(post_attributes)

                post_description = sub_soup.find('section', id = 'postingbody')
                description_.append(post_description)

                city_names_.append(city)

                time.sleep(random.randint(2,3))
                count += 1
                
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            title.append(title_)
            price.append(price_)
            neighborhood.append(neighborhood_)
            attributes.append(attributes_)
            description.append(description_)
            city_names.append(city_names_)


        print("\n")
        print("{} complete!".format(city.title()))
        print(str(len(title_)) + " rows collected.")
        print("\n")
    return title, price, neighborhood, attributes, description, city_names

In [28]:
title = []

for _ in title15:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)

price = []

for _ in price15:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

neighborhood = []

for _ in neighborhood15:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)

attributes = []

for _ in attributes15:
    for __ in _:
        attributes.append(__)

description = []

for _ in description15:
    for __ in _:
        description.append(__)

city = []

for _ in city15:
    for __ in _:
        city.append(__)

In [4]:
df = pd.read_csv('motorcycle_toy_df')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14103 entries, 0 to 14102
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         14102 non-null  object
 1   price         13591 non-null  object
 2   neighborhood  14033 non-null  object
 3   attributes    14103 non-null  object
 4   description   14102 non-null  object
 5   city          14103 non-null  object
dtypes: object(6)
memory usage: 661.2+ KB


In [13]:
df.drop('Unnamed: 0.1.1', axis=1, inplace=True)

In [20]:
df.index.names = ['index']

""
0
1
2
3
4
...
14098
14099
14100
14101


In [25]:
df = df.replace(np.nan, '', regex=True)
for _ in range(len(df)):
    df.replace(df['title'][_], str(df['title'][_]).replace('''<span id="titletextonly">''', '').replace('''</span>''', ''), inplace=True)
    df.replace(df['price'][_], int((df['price'][_].replace('''</span>''', '').replace('''<span class="price">$''', ''))), inplace=True)
    df.replace(df['neighborhood'][_], str(df['neighborhood'][_].replace('''<small> (''', '').replace(''')</small>''', '')), inplace=True)
    df.replace(df['attributes'][_], str(df['attributes'][_].replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''', '').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '')), inplace=True )
    df.replace(df['description'][_], str((df['description'][_].replace('''<section id="postingbody">\n<div class="print-information print-qrcode-container">\n<p class="print-qrcode-label">QR Code Link to This Post</p>\n<div class="print-qrcode" data-location=''', '').replace('''></div>\n</div>''', '').replace('''\n''', ' ').replace('''<br/>''', '').replace('''</section>''', '').split('''.html" '''))[-1]), inplace=True)

,title,price,neighborhood,attributes,description,city
index,,,,,,
0,"<span id=""titletextonly"">2016 Honda CBR300r</s...","<span class=""price"">$2900</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,"<span id=""titletextonly"">BAD CREDIT, NO CREDIT...",NaN,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,"<span id=""titletextonly"">2017 Yamaha FZ 07 ver...","<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,"<span id=""titletextonly"">2010 Yellow Goldwing ...",NaN,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,"<span id=""titletextonly"">2006 Honda XR650L *Pr...","<span class=""price"">$3800</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/


In [64]:
df = df.replace('', np.nan, regex=True)
df['price'] = pd.to_numeric(df['price'])
df.head()

,title,price,neighborhood,attributes,description,city
index,,,,,,
0,2016 Honda CBR300r,2900.0,Auburn,"2016 honda cbr300r,condition:excellent,engine ...","2016 Honda CBR300r 6,186 miles as of listing a...",https://auburn.craigslist.org/
1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",NaN,FAYETTEVILLE,"all bikes,condition:excellent,fuel:gas,title ...","WE SHIP NATIONWIDE, FINANCE NATIONWIDE! YOU SE...",https://auburn.craigslist.org/
2,2017 Yamaha FZ 07 very low miles,5000.0,Near Montgomery,"2017 Yamaha FZ-07,condition:like new,engine di...","2017 Yamaha FZ-07 with only 1100 miles, bike i...",https://auburn.craigslist.org/
3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,NaN,"<small>\n (<a href=""https://www.google....","2010 Honda GL1800 CSC,condition:like new,engin...","2010 Yellow GL1800 CSC Viper LOOK!!! only 8,2...",https://auburn.craigslist.org/
4,2006 Honda XR650L *Price Drop*,3800.0,Auburn,"2006 honda xr650l,condition:excellent,engine d...","2006 Honda XR650L 8,000 miles. This number w...",https://auburn.craigslist.org/


In [50]:
quicklst = []
for _ in range(len(df.attributes)):
    quicklst.append((len(df['attributes'][_].split(','))))
    for idx, i in enumerate(quicklst):
        if i ==13:
            print(idx)

1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717
1717


1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969


2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717
2969
1717


4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717


4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717
2969
4885
1717


...

1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630


1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630


1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630


6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885
6630
1717
2969
4885


KeyboardInterrupt: 

In [70]:
df[20:40]

,title,price,neighborhood,attributes,description,city
index,,,,,,
20,2014 Harley-Davidson Tri Glide Trike,18900.0,Athens,"2014 Harley Tri Glide,VIN:1HD1MAL16EB851071,co...",2014 Harley-Davidson Tri Glide Vin# 1HD1MAL16...,https://bham.craigslist.org/
21,BMW K1600 GTL - excellent condition,12900.0,Vestavia,"2014 BMK K1600 GTL,VIN:WB1061203EZZ25082,condi...","BMW K1600 GTL (MY 2014) 22,000 miles Engine\t...",https://bham.craigslist.org/
22,2017 Suzuki GSX-R750 (U01909),9999.0,Southeast,"2017 Suzuki GSX-R750,VIN:JS1GR7MA6H2101909,fue...",<p>Phone: 678-263-7545</p> <h3>2017 Suzuki GSX...,https://bham.craigslist.org/
23,Harley-Davidson Street Glide Electra Glide-Low...,7500.0,Lynnville tn,"2008 harley davidson,condition:excellent,fuel:...",This is a very nice 2008 Harley-Davidson Elect...,https://bham.craigslist.org/
24,Indian Scout,14600.0,35242,"2019 indian scout,VIN:56KMSA008K3140133,condit...",This is really for someone wanting a new bike ...,https://bham.craigslist.org/
25,2004 Ducati 1000DS Multistrada,3500.0,Birmingham,"ducati multistrada,fuel:gas,transmission:manual",2004 Ducati 1000DS Multistrada 4000 miles Re...,https://bham.craigslist.org/
26,2006 Harley-Davidson Heritage Softail,5500.0,Lynnville tn,"2006 harley davidson,condition:like new,fuel:g...",This is a very nice 2006 Harley-Davidson Herit...,https://bham.craigslist.org/
27,2015 Honda PCX 150 cc,2200.0,Downtown Birmingham,"2015 2015 Honda PCX 150 cc,VIN:RLHKF1803FY0008...","Very low miles. Like new, but, for a narrow s...",https://bham.craigslist.org/
28,2015 Harley Davidson Street XG500 XG 500,3500.0,Muscadine,"2015 Harley Davidson XG500,VIN:1HD4NAA18FC5036...",2015 Harley Davidson Street XG500 with only 12...,https://bham.craigslist.org/


In [68]:
df['attributes'][2969].split(',')

['2014 2014 Aprilia Dorsoduro ABS',
 'VIN:ZD4SMC010ES000023',
 'condition:like new',
 'cryptocurrency ok',
 'delivery available',
 'engine displacement (CC):750',
 'fuel:gas',
 'odometer:17378',
 'paint color:white',
 'street legal',
 'title status:clean',
 'transmission:manual',
 'type:dual-sport<p class="attrgroup">\n<span class="otherpostings">\n<a href="//inlandempire.craigslist.org/search/sss?userid=319057509">\nmore ads  by this user        </a>\n</span>\n</p>]']

In [79]:
model = []
vin = []
condition = []
cryptocurrency = []
delivery = []
engine_size = []
fuel_type = []
mileage = []
paint_color = []
street_legal = []
title_status = []
transmission = []
bike_type = []
bike_attributes = (df['attributes'][0].split(','))
# for idx_, _ in enumerate(bike_attributes):
    model.append(bike_attributes[0])
    if 'VIN:' in _:
        vin.append(_)
    else:
        vin.append('')
    if 'condition:' in _:
        condition.append(_)
    else:
        condition.append('')
    if 'cryptocurrency' in _:
        cryptocurrency.append(_)
    else:
        cryptocurrency.append('')
    if 'delivery' in _:
        delivery.append(_)
    else:
        delivery.append('')
    if 'engine displacement:' in _:
        engine_size.append(_)
    else:
        engine_size.append('')
    if 'fuel:' in _:
        fuel_type.append(_)
    else:
        fuel_type.append('')
    if 'odometer:' in _:
        mileage.append(_)
    else:
        mileage.append('')
    if 'paint color:' in _:
        paint_color.append(_)
    else:
        paint_color.append('')
    if 'street legal' in _:
        street_legal.append(_)
    else:
        street_legal.append('')
    if 'title status:' in _:
        title_status.append(_)
    else:
        title_status.append('')
    if 'transmission:' in _:
        transmission.append(_)
    else:
        transmission.append('')
    if 'type:' in _:
        bike_type.append(_)
    else:
        bike_type.append('')
    
    
    
# 2011 harley davidson 883 sportster
# VIN:1HD4CR210BC405214
# condition:like new
# cryptocurrency ok
# delivery available
# engine displacement (CC):883
# fuel:gas
# odometer:5914
# paint color:white
# street legal
# title status:clean
# transmission:manual
# type:touring

In [80]:
model, vin, condition, cryptocurrency, delivery, engine_size, fuel_type, mileage, paint_color, street_legal, title_status, transmission, bike_type 

(['2016 honda cbr300r'],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['type:sport bike'])

In [35]:

df['attributes'][0].replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''', '').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '')

'2016 honda cbr300r,condition:excellent,engine displacement (CC):286,fuel:gas,paint color:yellow,street legal,title status:clean,transmission:manual,type:sport bike'

In [45]:
df.replace(df['attributes'][0], df['attributes'][0].lstrip('''</b></span>\n<br/>\n</p>'''), inplace=True)
df['attributes'][1]

'[<p class="attrgroup">\n<span><b> all bikes</b></span>\n<br/>\n</p>, <p class="attrgroup">\n<span>condition: <b>excellent</b></span>\n<br/>\n<span>fuel: <b>gas</b></span>\n<br/>\n<span>title status: <b>clean</b></span>\n<br/>\n<span>transmission: <b>manual</b></span>\n<br/>\n</p>, <p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]'

In [53]:
(df['description'][0].replace('''<section id="postingbody">\n<div class="print-information print-qrcode-container">\n<p class="print-qrcode-label">QR Code Link to This Post</p>\n<div class="print-qrcode" data-location=''', '').replace('''></div>\n</div>''', '').replace('''\n''', ' ').replace('''<br/>''', '').replace('''</section>''', '').split('''.html" '''))[-1]

'2016 Honda CBR300r 6,186 miles as of listing after market: fender eliminator kit and bubble windscreen I have the original parts if you would prefer those. clean title  This is an excellent starter bike or daily commuter. 50+ mpg around auburn and the best parking on campus!  Not looking to trade  '

In [16]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,price,neighborhood,attributes,description,city
0,0,0,0,2016 Honda CBR300r,"<span class=""price"">$2900</span>",Auburn,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1,1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2,2,2017 Yamaha FZ 07 very low mile,"<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3,3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4,4,2006 Honda XR650L *Price Drop*,"<span class=""price"">$3800</span>",Auburn,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
